In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForTokenClassification, DistilBertConfig, AdamW, DistilBertModel
import pytorch_lightning as pl

import pandas as pd
import numpy as np
import os

/home/vri/miniconda3/envs/nlp_env/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
DATA_PATH = '../data/VUA18/'

In [3]:
train_df = pd.read_csv(DATA_PATH+'train.tsv', sep='\t', encoding='utf-8')
test_df = pd.read_csv(DATA_PATH+'test.tsv', sep='\t', encoding='utf-8')

In [4]:
print(f"The number of training samples is: {len(train_df)} and the number of test samples is: {len(test_df)}")

The number of training samples is: 97658 and the number of test samples is: 43947


In [5]:
train_df.head()

,index,label,sentence,POS,w_index
0,b1g-fragment02 841,0,If it now seems self-evident that monitoring o...,ADP,42
1,fef-fragment03 667,0,Which equation should we use in a practical ca...,NOUN,10
2,as6-fragment01 76,0,It was initiated partly in response to the fur...,ADP,10
3,ew1-fragment01 108,1,You fully know as an old pressman the difficul...,VERB,10
4,fpb-fragment01 1152,0,It was a condition of her gift to you the ten ...,ADJ,5


In [6]:
class MetaphorDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.data = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx) -> dict:
        sentence = self.data.iloc[idx]['sentence']
        label = self.data.iloc[idx]['label']
        pos_tag = self.data.iloc[idx]['POS']

        sentence_encoding = self.tokenizer(sentence, 
                                           truncation = True, 
                                           padding='max_length', 
                                           max_length=self.max_length,
                                           return_tensors= 'pt')
        pos_encoding = self.tokenizer(pos_tag,
                                      truncation = True,
                                      padding = 'max_length',
                                      max_length = 5,
                                      return_tensors= 'pt')
        
        return {
            'input_ids': sentence_encoding['input_ids'],
            'attention_mask': sentence_encoding['attention_mask'],
            'pos_ids': pos_encoding['input_ids'],
            'label': torch.tensor(label, dtype = torch.long)
        }

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')


In [8]:
max_length = 128  # You can adjust this based on your dataset
batch_size = 32   # You can adjust this as well
train_dataset = MetaphorDataset(train_df, tokenizer, max_length)
test_dataset = MetaphorDataset(test_df, tokenizer, max_length)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
len(test_loader)

1374